In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re 
import time 
import pandas as pd
from dotenv import load_dotenv
import os
import re
from datetime import datetime, timedelta

In [38]:
driver = webdriver.Chrome()

url = 'https://twitter.com/search?q=Reliance%20Industries%20Ltd.&src=typed_query&f=live'
driver.get(url)

In [39]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [40]:
time_lst = []
content_lst = []

def handle_index(string_list):
    for index, string in enumerate(string_list):
        if '@' in string:
            return index
    return None

while(len(content_lst)<200):
    driver.execute_script("window.scrollBy(0, 2 * window.innerHeight)")
    time.sleep(2)
    
    div_elements = driver.find_elements(By.CSS_SELECTOR , 'div.css-175oi2r.r-1iusvr4.r-16y2uox.r-1777fci.r-kzbkwu')
    # time.sleep(1)
    for i in div_elements:
    # Extracting time data
        twitter_text_split = i.text.split('\n')
        if(handle_index(twitter_text_split)!= None):
            time_idx = handle_index(twitter_text_split)+2
            time_lst.append(twitter_text_split[time_idx])
            content_str = ''
            for j in twitter_text_split[time_idx+1:-1]:
                content_str = content_str + " " + j
            content_lst.append(content_str)
    else:
        time_lst.append(None)
        content_lst.append(None)
    print(len(content_lst) , time_lst)
    

1 [None]
2 [None, None]
3 [None, None, None]
4 [None, None, None, None]
5 [None, None, None, None, None]
6 [None, None, None, None, None, None]
7 [None, None, None, None, None, None, None]
13 [None, None, None, None, None, None, None, '14h', '18h', 'In this Videobite, Patrick Park, Business Development Manager at UPM Raflatac, explains the company’s views of the label and package printing industry. The industry currently has serval hot topics from automation to product innovation and much more.', '21h', '22h', None]
23 [None, None, None, None, None, None, None, '14h', '18h', 'In this Videobite, Patrick Park, Business Development Manager at UPM Raflatac, explains the company’s views of the label and package printing industry. The industry currently has serval hot topics from automation to product innovation and much more.', '21h', '22h', None, '14h', '18h', 'In this Videobite, Patrick Park, Business Development Manager at UPM Raflatac, explains the company’s views of the label and packa

In [41]:
df = pd.DataFrame({'Content':content_lst , 'Time':time_lst})
df

,Content,Time
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
200,AMBANI AND RELIANCE LINK FOUND: Reliance Ind...,Mar 14
201,Replying to @s_b_tiwari and @PMishra_Journ...,Mar 14
202,Replying to @Mr_Unomi Reliance Industries (Q...,Mar 14
203,Replying to @Abhinav_Pan AMBANI AND RELIANCE...,Mar 14


In [42]:
df.to_csv('Twitter_unfiltered.csv')

Processing Time (Filtering posts within 24 hours)

In [54]:
# Function to convert different time formats to hours
def convert_to_hours(time_str):
    if (time_str!= None and len(time_str)<10 and any(char.isdigit() for char in time_str)):
        if 'm' in time_str:
            return int(time_str.replace('m', '')) / 60
        elif 'h' in time_str:
            return int(time_str.replace('h', ''))
        elif 'Mar' in time_str:
            temp_split = time_str.split(' ')
            return 24*(16-int(temp_split[-1]))
        else:
            return None
    else: 
        return 25

# Convert time column to hours
df['hours'] = df['Time'].apply(convert_to_hours)

# Filter rows with time <= 24 hours
filtered_df = df[df['hours'] <= 24]

print(filtered_df)

                                               Content    Time  hours
7     Replying to  @sidd_sharma01  @RenukaJain6  an...     14h     14
8     Nice to know! I still want to see 15% of Purc...     18h     18
10    AMBANI & RELIANCE LINK FOUND  Reliance Indust...     21h     21
11    Replying to  @SushantBSinha AMBANI AND RELIAN...     22h     22
13    Replying to  @sidd_sharma01  @RenukaJain6  an...     14h     14
..                                                 ...     ...    ...
180   Replying to  @ggganeshh El Bond list Quick Su...  Mar 15     24
181   AMBANI AND RELIANCE LINK FOUND:  Reliance Ind...  Mar 15     24
182   Replying to  @atullondhe AMBANI AND RELIANCE ...  Mar 15     24
187   AMBANI AND RELIANCE LINK FOUND:  Reliance Ind...  Mar 15     24
188   Replying to  @atullondhe AMBANI AND RELIANCE ...  Mar 15     24

[140 rows x 3 columns]


### Filter out unique rows based on the 'content' column

In [56]:
unique_df = filtered_df.drop_duplicates(subset=['Content'])
unique_df.to_csv('Twitter_filtered_unique.csv')